In [1]:
from judge_utils import *


/home/r3lativo/miniconda3/envs/mllms-con/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
results_data = load_all_results(results_path)

In [3]:
# Drop unnecessary columns since it's all the same
to_drop = ["Model",	"Quantization",	"Device",	"Number of models",	"Max new tokens",	"Repetition Penalty",	"Max rounds"]
results_data = results_data.drop(columns=to_drop).sort_values('run_time').reset_index(drop=True)

In [4]:
results_data.head()

,run_time,structure_id,use_img,use_json,shot,num_rounds,total_time_min,finished_by_architect,json_file
0,2025-02-11-1829-56,C1_bell,True,False,one-shot,5,1.0,True,2025-02-11-1829-56_C1_bell.json
1,2025-02-11-1830-50,C1_bell,True,False,zero-shot,15,90.0,False,2025-02-11-1830-50_C1_bell.json
2,2025-02-11-2008-08,C1_bell,True,True,one-shot,50,6.0,False,2025-02-11-2008-08_C1_bell.json
3,2025-02-11-2014-53,C1_bell,True,True,zero-shot,11,1.0,True,2025-02-11-2014-53_C1_bell.json
4,2025-02-11-2016-48,C1_bell,False,True,one-shot,50,13.0,False,2025-02-11-2016-48_C1_bell.json


In [ ]:
import numpy as np

def run_judge(command, results_df):

  # Name of the output JSON file
  output_file = f"judge_analysis_{command}.json"

  # Check if output JSON file exists; if so, load existing results, else create an empty list.
  if os.path.exists(output_file):
    with open(output_file, "r") as f:
      results = json.load(f)
  else:
    results = []

  # Iterate over each row in the DataFrame
  for _, row in results_df.iterrows():
    
    # Open and load the JSON file
    json_path = os.path.join(results_path, row["json_file"])
    parsed_conversation, _ = extract_conversation_data(json_path)
    
    # Process the JSON data with ask_judge function
    judge_output = ask_judge(command, parsed_conversation)
    rating = "Undefined"

    try:
      rating = int(judge_output[-1])
    except:
      pass
    
    # Create a dictionary with desired columns and the judge result
    result = {
      'structure_id': row['structure_id'],
      'use_img': row['use_img'],
      'use_json': row['use_json'],
      'shot': row['shot'],
      'judge': judge_output,
      'rating': rating,
    }
    
    # Append the result to our list
    results.append(result)
    
    # Update the JSON file with the new results
    with open(output_file, "w") as f:
      json.dump(results, f, indent=4)


In [6]:
run_judge("BASE", results_data)

2

1


KeyboardInterrupt: 

In [ ]:
def analyze_run(command, row):
  json_path = os.path.join(results_path, row["json_file"])
  parsed_conversation, _ = extract_conversation_data(json_path)
  evaluation = ask_judge(command, parsed_conversation)

  return row["structure_id", "use_img", "use_json", "shot"]
  try:
    return int(evaluation)
  except ValueError:
    return "Not a number"
  

In [4]:
results_data["BASE"] = results_data["json_file"].apply(lambda x: analyze_run("BASE", x))

In [5]:
results_data["CLAR_Q"] = results_data["json_file"].apply(lambda x: analyze_run("CLAR_Q", x))

In [6]:
results_data["COMM_SH_REF"] = results_data["json_file"].apply(lambda x: analyze_run("COMM_SH_REF", x))

In [7]:
results_data["IMPL_REF"] = results_data["json_file"].apply(lambda x: analyze_run("IMPL_REF", x))

In [8]:
results_data

,run_time,structure_name,json_file,Model,Quantization,Device,Number of models,Max new tokens,Repetition Penalty,Max rounds,use_img,use_json,num_rounds,total_round_time_min,finished_by_architect,BASE,CLAR_Q,COMM_SH_REF,IMPL_REF
0,2025-02-12-1445-39,C15_double_stairs,2025-02-12-1445-39_C15_double_stairs.json,llava-hf/llava-v1.6-mistral-7b-hf,4-bit,cuda,2,2048,1.1,50,True,True,17,104.0,False,Not a number,Not a number,Not a number,Not a number
1,2025-02-11-1829-56,C1_bell,2025-02-11-1829-56_C1_bell.json,llava-hf/llava-v1.6-mistral-7b-hf,4-bit,cuda,2,2048,1.1,50,True,False,5,1.0,True,2,Not a number,Not a number,Not a number
